In [6]:
import pandas as pd
import plotly.express as px
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [7]:
train = pd.read_csv('./data-titanic/train.csv')
test = pd.read_csv('./data-titanic/test.csv')

In [8]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [62]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
 12  Sex_b        891 non-null    int64  
dtypes: float64(2), int64(6), object(5)
memory usage: 90.6+ KB


In [63]:
train['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [64]:
def emb_n(x):
    if x == 'S':
        return 1
    if x == 'C':
        return 2
    else:
        return 3

In [65]:
train['Embarked_n'] = train['Embarked'].map(emb_n)
test['Embarked_n'] = test['Embarked'].map(emb_n)

In [66]:
train['Embarked_n'].value_counts()

1    644
2    168
3     79
Name: Embarked_n, dtype: int64

In [9]:
def sex_b(x):
    if x == 'female':
        return 1
    else:
        return 0 

train['Sex_b'] = train['Sex'].map(sex_b)
test['Sex_b'] = test['Sex'].map(sex_b)

In [129]:
v = ['Sex_b', 'Pclass','Parch','SibSp','Age'] # piorou o modelo 'Embarked_n'

In [130]:
x = train[v]
y = train['Survived']

In [131]:
# substituição dosvalores faltantes 
x = x.fillna(-1)

In [132]:
modelo = RandomForestClassifier(n_estimators=100, n_jobs=1, random_state=0)

In [133]:
# criar função para retornar perfomace com alteração de estimators (10-100)
# criar função para alterar variaveis encontrando as com mais peso em previsões

In [134]:
np.random.seed(1)
X_train, X_valid, y_train, y_valid = train_test_split(x, y, test_size=0.5)

In [135]:
X_train.head()

,Sex_b,Pclass,Parch,SibSp,Age
394,1,3,2,0,24.00
851,0,3,0,0,74.00
373,0,1,0,0,22.00
523,1,1,1,0,44.00
78,0,2,2,0,0.83


In [136]:
modelo.fit(X_train, y_train)

RandomForestClassifier(n_jobs=1, random_state=0)

In [137]:
p = modelo.predict(X_valid)

In [138]:
np.mean(y_valid == p)

0.7780269058295964

In [139]:
p = (X_valid['Sex_b'] == 1).astype(np.int64)
np.mean(y_valid == p)

0.7623318385650224

In [140]:
def best_estimators(X_train, X_valid, y_train, y_valid, n1=5, n2=50):
    valores = []
    np.random.seed(1)
    for i in range(n1, n2):
        modelo = RandomForestClassifier(n_estimators=i, random_state=0,)
        modelo.fit(X_train, y_train)
        p = modelo.predict(X_valid)
        valores.append(np.mean(y_valid == p))

    return valores       

In [141]:
list = best_estimators(X_train, X_valid, y_train, y_valid, 5, 100) #estimators n 5-50

In [142]:
px.line(list)